In [ ]:
import openai

client = openai.OpenAI()

message = []

In [ ]:
def call_ai():
    response = client.chat.completions.create(model='gpt-5-nano', messages = message)
    message = response.choices[0].message.content
    message.append({"role": "assistant", "content": message})

    print(f"AI: {message}")

In [ ]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        message.append({
            "role": "user",
            "content": message
        })
        print(f"User: {message}")
        call_ai()